In [3]:
# Sel 1: Setup dan Import Dependencies
%pip install torch torchvision
!git clone https://github.com/ultralytics/yolov5

import sys, torch
sys.path.append('..')
from models.backbones.efficientnet_backbone import EfficientNetBackbone

from utils.logger import SmartCashLogger

logger = SmartCashLogger(__name__)

Note: you may need to restart the kernel to use updated packages.
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [5]:
# Sel 2: Inisialisasi Backbone
backbone = EfficientNetBackbone(
    logger=logger,
    pretrained=True,
    trainable_layers=3  # 3 block terakhir bisa di-fine-tune
)


2025-02-17 17:51:43 - ℹ️ 💻 Menggunakan CPU untuk komputasi
2025-02-17 17:51:43 - ℹ️ 💻 Menggunakan CPU untuk komputasi
2025-02-17 17:51:43 - ℹ️ 🔄 Memuat pre-trained EfficientNet-B4...
2025-02-17 17:51:43 - ℹ️ 🔄 Memuat pre-trained EfficientNet-B4...


Using cache found in /Users/masdevid/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/Users/masdevid/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/Users/masdevid/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


2025-02-17 17:51:43 - ✅ ✨ Model berhasil dimuat
2025-02-17 17:51:43 - ✅ ✨ Model berhasil dimuat
2025-02-17 17:51:43 - ℹ️ 🎯 Menyiapkan fine-tuning:
   Total blocks: 4
   Trainable blocks: 3
2025-02-17 17:51:43 - ℹ️ 🎯 Menyiapkan fine-tuning:
   Total blocks: 4
   Trainable blocks: 3
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: layers
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: layers
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: features
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: features
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: classifier
2025-02-17 17:51:43 - ℹ️ 🔓 Mengaktifkan fine-tuning untuk block: classifier


In [7]:
# Sel 3: Analisis Parameter Fine-tuning
trainable_params = backbone.get_trainable_params()
logger.info("\n🔍 Parameter yang bisa di-fine-tune:")
for name, shape in trainable_params.items():
    logger.info(f"   {name}: {shape}")

2025-02-17 17:52:05 - ℹ️ 
🔍 Parameter yang bisa di-fine-tune:
2025-02-17 17:52:05 - ℹ️ 
🔍 Parameter yang bisa di-fine-tune:
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.conv.weight: torch.Size([48, 1, 3, 3])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.conv.weight: torch.Size([48, 1, 3, 3])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.bn.weight: torch.Size([48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.bn.weight: torch.Size([48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.bn.bias: torch.Size([48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.depsep.bn.bias: torch.Size([48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.se.squeeze.weight: torch.Size([12, 48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.se.squeeze.weight: torch.Size([12, 48])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.se.squeeze.bias: torch.Size([12])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.se.squeeze.bias: torch.Size([12])
2025-02-17 17:52:05 - ℹ️    layers.0.block0.se.expand.weight

In [8]:
# Sel 4: Test Forward Pass
def test_forward():
    logger.info("\n🧪 Testing forward pass...")
    
    # Buat dummy input (batch_size=2, channels=3, height=640, width=640)
    x = torch.randn(2, 3, 640, 640)
    
    # Pindahkan ke device yang sesuai
    x = x.to(backbone.device)
    backbone.to(backbone.device)
    
    # Forward pass
    with torch.no_grad():
        features = backbone(x)
        
    # Print info feature maps
    logger.info("\n📊 Feature Maps Info:")
    for i, feat in enumerate(features):
        logger.info(
            f"   Level {i+1}: shape={feat.shape}, "
            f"mean={feat.mean():.3f}, std={feat.std():.3f}"
        )
        
test_forward()

2025-02-17 17:52:28 - ℹ️ 
🧪 Testing forward pass...
2025-02-17 17:52:28 - ℹ️ 
🧪 Testing forward pass...
2025-02-17 17:52:30 - ℹ️ 
📊 Feature Maps Info:
2025-02-17 17:52:30 - ℹ️ 
📊 Feature Maps Info:


In [9]:
# Sel 5: Visualisasi Feature Maps
import matplotlib.pyplot as plt

def visualize_feature_maps(features, max_channels=3):
    """
    Visualisasi feature maps dari backbone
    Args:
        features: List feature maps dari model
        max_channels: Jumlah channel yang akan divisualisasikan
    """
    plt.figure(figsize=(15, 5))
    
    for i, feat in enumerate(features):
        feat = feat[0].cpu()  # Ambil batch pertama
        
        # Plot beberapa channel
        for j in range(min(max_channels, feat.shape[0])):
            plt.subplot(len(features), max_channels, i*max_channels + j + 1)
            plt.imshow(feat[j].numpy())
            plt.axis('off')
            plt.title(f'Level {i+1} Ch{j+1}')
            
    plt.tight_layout()
    plt.show()

# Test dengan dummy image
test_img = torch.randn(1, 3, 640, 640).to(backbone.device)
with torch.no_grad():
    features = backbone(test_img)
visualize_feature_maps(features)

<Figure size 1500x500 with 0 Axes>